In [0]:
!pip install geopy

In [0]:
import pandas as pd
import requests
import io
import time
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt

# --- CONFIGURATION ---
INPUT_FILE = "New_Query_2025_12_09_3_08pm.csv" # file that stores all the cities of interest, could be a table too
OUTPUT_FILE = "verified_stations_with_data.csv"
START_YEAR = 1990
END_YEAR = 2025
spark.sql("CREATE TABLE IF NOT EXISTS main.ext.weather") # creating the table that holds all the weather information

In [0]:
def haversine(lon1, lat1, lon2, lat2):
    """Calculate distance (km) between two points."""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return c * 6371

In [0]:
def get_station_inventory():
    print("Downloading Station Inventory...")
    url = "https://collaboration.cmc.ec.gc.ca/cmc/climate/Get_More_Data_Plus_de_donnees/Station%20Inventory%20EN.csv"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        return pd.read_csv(io.StringIO(r.content.decode('utf-8', errors='ignore')), skiprows=3)
    except Exception as e:
        print(f"Inventory Download Failed: {e}")
        return pd.DataFrame()

In [0]:
def get_sample_temp(station_id, test_year):
    """
    Downloads data for a test year and returns a sample valid temperature.
    Returns: (is_valid, sample_text)
    """
    base_url = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html"
    params = {
        'format': 'csv', 
        'stationID': station_id, 
        'Year': test_year, 
        'Month': 1, 'Day': 1, 
        'timeframe': 2, 
        'submit': 'Download Data'
    }
    try:
        r = requests.get(base_url, params=params, timeout=10)
        content = r.content.decode('utf-8')
        
        # Dynamic Header Detection
        lines = content.split('\n')
        skip_rows = 0
        for i, line in enumerate(lines):
            if "Date/Time" in line:
                skip_rows = i
                break
        
        if skip_rows == 0 and "Date/Time" not in lines[0]:
            return False, "Format Error"

        df = pd.read_csv(io.StringIO(content), skiprows=skip_rows)
        
        # Check for Max Temp column
        max_temp_col = next((c for c in df.columns if "Max Temp" in c), None)
        # Check for Min Temp column
        min_temp_col = next((c for c in df.columns if "Min Temp" in c), None)
        
        if max_temp_col:
            # Drop Nulls and get a sample
            valid_data = df.dropna(subset=[max_temp_col])
            #print("Printing Valid Data")
            #print(valid_data)
            count = len(valid_data)
            all_values =[]
            if not valid_data.empty:
                #print("Counting")
                #print(valid_data)
                # Pick a random valid row to display
                example = valid_data.iloc[0]
                example_date = example['Date/Time']
                max_temp = float(example[max_temp_col])
                i = 0
                for i in range(count):
                    sample = valid_data.iloc[i]
                    date = sample['Date/Time']
                    max_temp = float(sample[max_temp_col])
                    min_temp = float(sample[min_temp_col])
                    all_values.append({
                            "Station_ID": station_id,
                            "Date": date,
                            "Max_Temp": max_temp,
                            "Min_Temp": min_temp
                        })
                    i = i + 1
                #print("Printing next data point")
                #print(valid_data.iloc[1])
                return True, all_values, max_temp, example_date
            else:
                return False, None, None, None
        else:
            return False, None, None, None
        
    except Exception as e:
        return False, None, None, None

In [0]:
def find_verified_station(lat, lon, inventory_df):
    """
    Finds nearest station and VERIFIES it has temperature data by looking it up.
    """
    if pd.isna(lat) or pd.isna(lon): return None, None, "No Coords", 0

    # Filter for active years
    inv = inventory_df.copy()
    inv['DLY First Year'] = pd.to_numeric(inv['DLY First Year'], errors='coerce')
    inv['DLY Last Year'] = pd.to_numeric(inv['DLY Last Year'], errors='coerce')
    
    candidates = inv[
        (inv['DLY Last Year'] >= START_YEAR) & 
        (inv['DLY First Year'] <= END_YEAR)
    ].copy()

    if candidates.empty: return None, None, "No Active Stations", 0

    # Calculate distance
    lat_col = [c for c in inv.columns if "Latitude" in c][0]
    lon_col = [c for c in inv.columns if "Longitude" in c][0]
    candidates['distance'] = candidates.apply(
        lambda row: haversine(lon, lat, row[lon_col], row[lat_col]), axis=1
    )
    candidates = candidates.sort_values('distance')
    
    # Check top 5 closest stations
    for index, row in candidates.iloc[:5].iterrows():
        st_id = row['Station ID']
        st_name = row['Name']
        
        # Pick a test year (middle of our range or last active year)
        test_year = int((START_YEAR + END_YEAR) / 2)
        if test_year > row['DLY Last Year']: test_year = int(row['DLY Last Year'])
        
        # PROBE THE DATA
        is_valid, results, sample_text, sample_date = get_sample_temp(st_id, test_year)
        #print("Printing getting sample")
        #print(get_sample_temp(st_id, test_year))
        if is_valid:
            #print(results)
            return st_id, st_name, sample_text, sample_date, row['distance'], results
        else:
            # print(f"    x Skipped {st_name} (ID: {st_id}) - {sample_text}")
            time.sleep(0.2)

    return None, None, None, None, 0, None

In [0]:
# --- MAIN EXECUTION ---
if __name__ == "__main__":
    df_locs = spark.sql("select distinct County from main.silver_raw.monthlymarketshare_details").toPandas()
    inventory = get_station_inventory()
    geolocator = Nominatim(user_agent="weather_display_v3")
    
    results = []
    
    for year in range(START_YEAR, END_YEAR + 1):
        for idx, row in df_locs.iterrows():
            loc_name = row['County']
            
            try:
                # 1. Geocode
                loc = geolocator.geocode(f"{loc_name}, Canada", timeout=10)
                if loc:
                    # 2. Find & Verify
                    st_id, st_name, sample, date, dist, final_results = find_verified_station(loc.latitude, loc.longitude, inventory)
                    #print(st_id)
                    if st_id:
                        i = 0
                        for i in range(len(final_results)):
                            results.append({
                                "Original_Location": loc_name,
                                "Station_ID": st_id,
                                "Station_Name": st_name,
                                "Max_Temp": final_results[i]['Max_Temp'],
                                "Min_Temp": final_results[i]['Min_Temp'],
                                "Date": final_results[i]['Date']                                
                                })
                            results_df = pd.DataFrame(results)
                            spark_df = spark.createDataFrame(results_df)
                            spark_df.write.format("delta") \
                                    .mode("append") \
                                    .option("mergeSchema", "true") \
                                    .saveAsTable("main.ext.weather")
                            i = i + 1
                    else:
                        print(f"{loc_name[:30]:<30} | {'No Match':<25} | -          | {sample}")
                else:
                    print(f"{loc_name[:30]:<30} | {'Geo Failed':<25} | -          | -")
                    
            except Exception as e:
                print(f"Error processing {loc_name}: {e}")
            
            # Rate limit
            time.sleep(1)